<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load-the-libraries" data-toc-modified-id="Load-the-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="#Missing-values" data-toc-modified-id="Missing-values-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Missing values</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Script" data-toc-modified-id="Script-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Script</a></span></li></ul></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Train-validation-split" data-toc-modified-id="Train-validation-split-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Train validation split</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Xgboost" data-toc-modified-id="Xgboost-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Xgboost</a></span></li></ul></li></ul></div>

# Description
Reference: https://datahack.analyticsvidhya.com/contest/all/  


**Predict Loan Eligibility for Dream Housing Finance company**
Dream Housing Finance company deals in all kinds of home loans. They have presence across all urban, semi urban and rural areas. Customer first applies for home loan and after that company validates the customer eligibility for loan.

Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have provided a dataset to identify the customers segments that are eligible for loan amount so that they can specifically target these customers. 

**Data Dictionary**
Train file: CSV containing the customers for whom loan eligibility is known as 'Loan_Status'

| Variable | Description |
| :---|:---|
| Loan_ID | Unique Loan ID |
| Gender | Male/ Female |
| Married | Applicant married (Y/N) |
| Dependents | Number of dependents |
| Education | Applicant Education (Graduate/ Under Graduate) |
| Self_Employed | Self employed (Y/N) |
| ApplicantIncome | Applicant income |
| CoapplicantIncome | Coapplicant income |
| LoanAmount | Loan amount in thousands |
| Loan_Amount_Term | Term of loan in months |
| Credit_History | credit history meets guidelines |
| Property_Area | Urban/ Semi Urban/ Rural |
| Loan_Status | (Target) Loan approved (Y/N) |


**Evaluation Metric**  
Your model performance will be evaluated on the basis of your prediction of loan status for the test data (test.csv), which contains similar data-points as train except for the loan status to be predicted. Your submission needs to be in the format as shown in sample submission.

We at our end, have the actual loan status for the test dataset, against which your predictions will be evaluated. We will use the Accuracy value to judge your response.



**Public and Private Split**   
Test file is further divided into Public (25%) and Private (75%)

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

# Load the libraries

In [84]:
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.plotting.backend = "matplotlib"
pd.set_option('max_columns',100)

import time,os,json
time_start_notebook = time.time()
home = os.path.expanduser('~')
SEED=100

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

[(x.__name__,x.__version__) for x in [np,pd,sns]]

[('numpy', '1.18.4'), ('pandas', '1.1.0'), ('seaborn', '0.10.1')]

In [85]:
import sklearn
import xgboost as xgb

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
df_eval = pd.DataFrame({
    'Model': [],
    'Description': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F-score': []
})

# Load the data

In [88]:
def clean_data(df):
    df = df.copy()
    # drop unwanted features
    df = df.drop('Loan_ID',axis=1)
    
    ## try fillnan = unknown for married
    ## I got same accuracy for xgb9.0
#     df['Married'] = df['Married'].fillna(2)

    ## try fillnan = unknown for self_employed
    ## I got same accuracy for xgb9.0
    df['Self_Employed'] = df['Self_Employed'].fillna(2)

    # missing values imputation
    ## fill with mode
    cols_mode = [
        'Married',
        'Gender',
        'Dependents',
#         'Self_Employed',
        'Credit_History']
    for c in cols_mode:
        df[c] = df[c].fillna(df[c].mode()[0])

    ## fill with frequency
    ## BAD!!
#     for c in cols_mode:
#         np.random.seed(SEED)
#         s = df[c].value_counts(normalize=True)
#         val = np.random.choice(s.index, p=s.values,
#                                size=df[c].isna().sum())
#         df.loc[df[c].isna(), c] = val

    ##=====================================
    ## fill with mean
    cols_mean = ['LoanAmount',
                 'Loan_Amount_Term'
                ]

    for c in cols_mean:
        df[c] = df[c].fillna(df[c].mean())

    ## fill with mean with more columns
    ## BAD for xgb9.0
#     other_cols = [
#         'Gender',
#         'Dependents',
#         'Education',
#         'ApplicantIncome',
#         'Married',     
#         'Property_Area',
#         'Self_Employed']

#     for c in cols_mean: 
#         df[c] = df.groupby(other_cols)[c].transform(
#                      lambda x: x.fillna(x.mean()))
    ##==============================================
    ## Try loan term binning
    ## This gave me same acc for xgb9.0 
#     df['Loan_Amount_Term_Very_Short'] = df['Loan_Amount_Term'].map(
#         lambda t: 1 if t<=60 else 0)
#     df['Loan_Amount_Term_Short'] = df['Loan_Amount_Term'].map(
#         lambda t: 1 if t>60 and t<180 else 0)
#     df['Loan_Amount_Term_Long'] = df['Loan_Amount_Term'].map(
#         lambda t: 1 if t>=180 and t<=300  else 0)
#     df['Loan_Amount_Term_Very_Long'] = df['Loan_Amount_Term'].map(
#         lambda t: 1 if t>300 else 0)
#     df.drop('Loan_Amount_Term', axis=1, inplace=True)        
        


    # mapping string to integers
    df['Gender'] = df['Gender'].map({'Male':1, 'Female': 0})
    df['Married'] = df['Married'].map({'Yes':1, 'No': 0 })
    df['Education'] = df['Education'].map({'Graduate': 1, 'Not Graduate': 0})
    df['Self_Employed'] = df['Self_Employed'].map({'Yes':1, 'No': 0})
    
    
    ## add features
    ## BAD
#     df['TotalIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']

#     df['DebtIncomeRatio'] = df['TotalIncome'] / df['LoanAmount']



    # data transformation
#     df['LoanAmount_log'] = np.log1p(df['LoanAmount'])
#     df = df.drop('LoanAmount',axis=1)
    
    # target 
    target = 'Loan_Status'
    if target in df.columns:
        df[target] = df[target].map({'Y':1, 'N': 0})

    ## try make dependents 3+ to number 3
    ## xgboost9.0 gives same result as OHE encoding
#     df['Dependents'] = df['Dependents'].str.rstrip('+').astype(int)
  
    # one hot encoding
    cols = [
        'Dependents',
        'Property_Area'
    ]
    df = pd.get_dummies(df,columns=cols,drop_first=True)
    return df

##===============================================================
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')

df_train = clean_data(df_train)
df_test = clean_data(df_test)
df_train_orig = df_train.copy()
target = 'Loan_Status'
df_Xtrain, df_Xvalid, ser_ytrain, ser_yvalid = train_test_split(
    df_train_orig.drop(target,axis=1), df_train_orig[target],
    test_size = 0.2,
    random_state=SEED,
    stratify=df_train_orig[target]
)

ytrain = ser_ytrain.to_numpy().ravel()
yvalid = ser_yvalid.to_numpy().ravel()
Xtr = df_Xtrain
ytr = ytrain
Xvd = df_Xvalid
yvd = yvalid

In [89]:
# default from xgboost version 0.9
model = XGBClassifier(
    # always same
    tree_method='auto',
    objective='binary:logistic',
    random_state=100,
    n_jobs=-1,
    verbosity=1,
    base_score=0.5,

    # most imp
    n_estimators=100,
    learning_rate=0.1,

    # subsample and colsample
    subsample=1,
    colsample_bylevel=1,
    colsample_bynode=1, 
    colsample_bytree=1,

    # tree depth
    max_depth=3,
    min_child_weight=1,
    scale_pos_weight=1,
    
    # regularizaion alpha lambda gamma
    reg_alpha=0,
    reg_lambda=1,
    gamma=0,
    )

model.fit(Xtr,ytr)
vd_preds = model.predict(Xvd)

acc = metrics.accuracy_score(yvd, vd_preds)
pre = metrics.precision_score(yvd, vd_preds)
rec = metrics.recall_score(yvd, vd_preds)
f1  = metrics.f1_score(yvd,vd_preds)

row = ['XGB', 'default from version 0.9', acc, pre, rec, f1]
df_eval.loc[len(df_eval)] = row
df_eval = df_eval.drop_duplicates(['Model','Description'])
df_eval.sort_values('Accuracy',ascending=False)

,Model,Description,Accuracy,Precision,Recall,F-score
0,XGB,default from version 0.9,0.829268,0.807692,0.988235,0.888889


In [90]:
"""
xgb9.0 default loanterm fill mean
0.829268

try1
0.813008 ==> Loan_Amount_Term fillna mode **BAD!!**

try2
0.821138 ==> added two new cols: totalincome and debtratio **BAD**

try3
0.821138 ==> fillnans with frequency
""";